# Block-Level Top Services Analysis
Generate top 6 services per block based on usage count

In [3]:
import pandas as pd
import os

# Set data directory
DATA_DIR = r"C:\SysReco\data"

print("Loading data...")
# Load the datasets
bsk_master = pd.read_csv(os.path.join(DATA_DIR, "ml_bsk_master.csv"), encoding='latin-1')
new_provision = pd.read_csv(os.path.join(DATA_DIR, "new_ml_provision.csv"), encoding='latin-1')
service_master = pd.read_csv(os.path.join(DATA_DIR, "service_master.csv"), encoding='latin-1')

print(f"✓ Loaded bsk_master: {bsk_master.shape}")
print(f"  Columns: {bsk_master.columns.tolist()}")
print(f"✓ Loaded new_provision: {new_provision.shape}")
print(f"✓ Loaded service_master: {service_master.shape}")

Loading data...
✓ Loaded bsk_master: (3561, 24)
  Columns: ['bsk_id', 'bsk_name', 'district_name', 'sub_division_name', 'block_municipalty_name', 'gp_ward', 'gp_ward_distance', 'bsk_type', 'bsk_sub_type', 'bsk_code', 'no_of_deos', 'is_aadhar_center', 'bsk_address', 'bsk_lat', 'bsk_long', 'bsk_account_no', 'bsk_landline_no', 'is_saturday_open', 'is_active', 'district_id', 'block_mun_id', 'gp_id', 'sub_div_id', 'pin']
✓ Loaded new_provision: (11486353, 9)
✓ Loaded service_master: (422, 15)


In [ ]:
# Merge provision with bsk_master to get block_id and block_mun_id
print("\nMerging data on bsk_id...")
merged_df = pd.merge(
    new_provision,
    bsk_master[['bsk_id', 'bsk_name', 'block_mun_id']],  # Include block_mun_id
    on='bsk_id',
    how='inner'
)

# Add service names
merged_df = pd.merge(
    merged_df,
    service_master[['service_id', 'service_name']],
    on='service_id',
    how='left'
)

print(f"✓ Merged data: {merged_df.shape}")
print(f"  Unique blocks: {merged_df['bsk_id'].nunique()}")
print(f"  Unique services: {merged_df['service_id'].nunique()}")
print(f"  Columns: {merged_df.columns.tolist()}")


Merging data on bsk_id...


KeyError: "['block_muni_id'] not in index"

In [ ]:
# Count service usage per block
print("\nCalculating service usage counts per block...")

# First check what columns we have
print("Available columns:", merged_df.columns.tolist())

# Check which key columns exist
has_bsk_id = 'bsk_id' in merged_df.columns
has_service_id = 'service_id' in merged_df.columns
has_service_name = 'service_name' in merged_df.columns
has_customer_id = 'customer_id' in merged_df.columns
has_block_mun_id = 'block_mun_id' in merged_df.columns

print(f"Has bsk_id: {has_bsk_id}")
print(f"Has service_id: {has_service_id}")
print(f"Has service_name: {has_service_name}")
print(f"Has customer_id: {has_customer_id}")
print(f"Has block_mun_id: {has_block_mun_id}")

# Build groupby columns based on what exists
groupby_cols = []
if has_bsk_id:
    groupby_cols.append('bsk_id')

# Find block name column
block_name_col = None
for col in merged_df.columns:
    if 'name' in col.lower() and 'bsk' in col.lower():
        block_name_col = col
        groupby_cols.append(block_name_col)
        print(f"Using '{block_name_col}' as block name column")
        break

if has_block_mun_id:
    groupby_cols.append('block_mun_id')

if has_service_id:
    groupby_cols.append('service_id')
    
if has_service_name:
    groupby_cols.append('service_name')

print(f"Grouping by: {groupby_cols}")

# Group and aggregate
block_service_counts = merged_df.groupby(groupby_cols).agg(
    usage_count=('customer_id', 'count'),
    unique_customers=('customer_id', 'nunique')
).reset_index()

# Add service_name if it wasn't in the original merge
if not has_service_name and 'service_id' in block_service_counts.columns:
    print("Adding service names from service_master...")
    block_service_counts = pd.merge(
        block_service_counts,
        service_master[['service_id', 'service_name']],
        on='service_id',
        how='left'
    )

# Rank services within each block by usage_count
block_service_counts['rank_in_block'] = block_service_counts.groupby('bsk_id')[
    'usage_count'
].rank(ascending=False, method='dense').astype(int)

print(f"✓ Calculated usage counts: {block_service_counts.shape}")
print(f"\nSample data:")
print(block_service_counts.head(10))


Calculating service usage counts per block...
Available columns: ['bsk_id', 'bsk_name_x', 'customer_id', 'customer_name', 'customer_phone', 'service_id', 'service_name_x', 'prov_date', 'docket_no', 'bsk_name_y', 'service_name_y']
Has bsk_id: True
Has service_id: True
Has service_name: False
Has customer_id: True
Using 'bsk_name_x' as block name column
Grouping by: ['bsk_id', 'bsk_name_x', 'service_id']
Adding service names from service_master...
✓ Calculated usage counts: (176439, 7)

Sample data:
   bsk_id             bsk_name_x  service_id  usage_count  unique_customers  \
0       1  DM OFFICE  ALIPURDUAR           1            1                 1   
1       1  DM OFFICE  ALIPURDUAR           2            2                 2   
2       1  DM OFFICE  ALIPURDUAR           6            1                 1   
3       1  DM OFFICE  ALIPURDUAR          22           92                70   
4       1  DM OFFICE  ALIPURDUAR         119            1                 1   
5       1  DM OFFICE  

In [4]:
# Filter top 6 services per block
print("\nFiltering top 6 services per block...")
top_6_per_block = block_service_counts[
    block_service_counts['rank_in_block'] <= 6
].copy()

# Sort by block_id and rank
top_6_per_block = top_6_per_block.sort_values(['bsk_id', 'rank_in_block'])

print(f"✓ Top 6 services per block: {top_6_per_block.shape}")
print(f"  Total blocks with data: {top_6_per_block['bsk_id'].nunique()}")

# Show example for first block
example_block = top_6_per_block['bsk_id'].iloc[0]
print(f"\nExample - Block {example_block}:")
display_cols = ['service_name', 'usage_count', 'rank_in_block']
if block_name_col and block_name_col in top_6_per_block.columns:
    display_cols = [block_name_col] + display_cols
print(top_6_per_block[top_6_per_block['bsk_id'] == example_block][display_cols])


Filtering top 6 services per block...
✓ Top 6 services per block: (22267, 7)
  Total blocks with data: 3559

Example - Block 1:
               bsk_name_x                                       service_name  \
22  DM OFFICE  ALIPURDUAR  Payment - e-Payment Stamp Duty & Registration ...   
7   DM OFFICE  ALIPURDUAR      Apply - Submission of Form for Electoral Roll   
3   DM OFFICE  ALIPURDUAR                      Apply - Land Revenue (Khajna)   
13  DM OFFICE  ALIPURDUAR        Apply - Payment of WBSEDCL Electricity Bill   
8   DM OFFICE  ALIPURDUAR      Apply - Verification of Electoral Roll Online   
9   DM OFFICE  ALIPURDUAR              Apply - Yuvashree Scheme Application    
12  DM OFFICE  ALIPURDUAR                                Apply - Map Service   

    usage_count  rank_in_block  
22          144              1  
7            98              2  
3            92              3  
13           35              4  
8            17              5  
9             6              6  

In [5]:
# Save to CSV
output_file = os.path.join(DATA_DIR, "block_top_services.csv")
top_6_per_block.to_csv(output_file, index=False, encoding='utf-8')

print(f"\n✅ Saved to: {output_file}")
print(f"\nColumns in output file:")
print(list(top_6_per_block.columns))

# Display summary statistics
print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"Total blocks: {top_6_per_block['bsk_id'].nunique()}")
print(f"Total records: {len(top_6_per_block)}")
print(f"Average services per block: {len(top_6_per_block) / top_6_per_block['bsk_id'].nunique():.1f}")

# Show blocks with fewer than 6 services
blocks_with_less_than_6 = top_6_per_block.groupby('bsk_id').size()
blocks_with_less_than_6 = blocks_with_less_than_6[blocks_with_less_than_6 < 6]
if len(blocks_with_less_than_6) > 0:
    print(f"\nBlocks with fewer than 6 services: {len(blocks_with_less_than_6)}")
    print(blocks_with_less_than_6.head())


✅ Saved to: C:\SysReco\data\block_top_services.csv

Columns in output file:
['bsk_id', 'bsk_name_x', 'service_id', 'usage_count', 'unique_customers', 'service_name', 'rank_in_block']

SUMMARY
Total blocks: 3559
Total records: 22267
Average services per block: 6.3

Blocks with fewer than 6 services: 15
bsk_id
7       5
21      4
381     5
421     5
1147    5
dtype: int64


In [6]:
# View sample output
print("\n" + "="*60)
print("SAMPLE OUTPUT - First 3 blocks")
print("="*60)

for block_id in top_6_per_block['bsk_id'].unique()[:3]:
    block_data = top_6_per_block[top_6_per_block['bsk_id'] == block_id]
    
    # Get block name if available
    if block_name_col and block_name_col in block_data.columns:
        block_name = block_data[block_name_col].iloc[0]
        print(f"\n📍 Block {block_id} - {block_name}")
    else:
        print(f"\n📍 Block {block_id}")
    
    print("-" * 60)
    for _, row in block_data.iterrows():
        print(f"  {row['rank_in_block']}. {row['service_name']} (Used {row['usage_count']} times by {row['unique_customers']} customers)")


SAMPLE OUTPUT - First 3 blocks

📍 Block 1 - DM OFFICE  ALIPURDUAR
------------------------------------------------------------
  1. Payment - e-Payment Stamp Duty & Registration Fees (Used 144 times by 39 customers)
  2. Apply - Submission of Form for Electoral Roll (Used 98 times by 73 customers)
  3. Apply - Land Revenue (Khajna) (Used 92 times by 70 customers)
  4. Apply - Payment of WBSEDCL Electricity Bill (Used 35 times by 24 customers)
  5. Apply - Verification of Electoral Roll Online (Used 17 times by 15 customers)
  6. Apply - Yuvashree Scheme Application  (Used 6 times by 6 customers)
  6. Apply - Map Service (Used 6 times by 4 customers)

📍 Block 2 - DM OFFICE BANKURA
------------------------------------------------------------
  1. Search - GRN Information (Used 364 times by 128 customers)
  2. Apply - Map Service (Used 279 times by 93 customers)
  3. Apply - Payment of WBSEDCL Electricity Bill (Used 255 times by 152 customers)
  4. Search - Status of Application of Swast